# Actividad Integradora

### Daniel Emilio Fuentes Portaluppi - A01708302

# Descripción
¡Felicidades! Eres el orgulloso propietario de 5 robots nuevos y un almacén lleno de cajas. El dueño anterior del almacén lo dejó en completo desorden, por lo que depende de tus robots organizar las cajas en algo parecido al orden y convertirlo en un negocio exitoso.

Cada robot está equipado con ruedas omnidireccionales y, por lo tanto, puede conducir en las cuatro direcciones. Pueden recoger cajas en celdas de cuadrícula adyacentes con sus manipuladores, luego llevarlas a otra ubicación e incluso construir pilas de hasta cinco cajas. Todos los robots están equipados con la tecnología de sensores más nueva que les permite recibir datos de sensores de las cuatro celdas adyacentes. Por tanto, es fácil distinguir si un campo está libre, es una pared, contiene una pila de cajas (y cuantas cajas hay en la pila) o está ocupado por otro robot. Los robots también tienen sensores de presión equipados que les indican si llevan una caja en ese momento.

Lamentablemente, tu presupuesto resultó insuficiente para adquirir un software de gestión de agentes múltiples de última generación. Pero eso no debería ser un gran problema ... ¿verdad? Tu tarea es enseñar a sus robots cómo ordenar su almacén. La organización de los agentes depende de ti, siempre que todas las cajas terminen en pilas ordenadas de cinco.

**Puntos a considerar**
* La semilla para generación de números aleatorios será 67890.
* El almacén es 20x20 celdas.
* Al inicio de la simulación, tu solución deberá colocar 200 cajas repartidas en grupos de 1 a 3 cajas en posiciones aleatorias.
* Todos los robots empiezan en posiciones aleatorias vacías. * Y, sólo puede haber un robot por celda.
* La simulación termina cuando todas las cajas se encuentra apiladas en pilas de exactamente 5 cajas.

**¿Que debes entregar?**
* Un cuaderno de Jupyter Notebook conteniendo un reporte de la actividad. El cuaderno deberá contener:
* Código fuente documentado.
* Descripción detallada de la estrategia y los mecanismos utilizados en tu solución.
* Una visualización que permita ver los diferentes pasos de la simulación.
* El número de pasos necesarios para terminar la simulación.
* ¿Existe una forma de reducir el número de pasos utilizados? Si es así, ¿cuál es la estrategia que se tendría en implementar?

**Criterios de evaluación**
<br>Los criterios que se utilizarán para evaluar sus soluciones y seleccionar a los tres primeros ganadores son los siguientes:
* Aplicación original, innovadora y efectiva de algoritmos computacionales para resolver problemas específicos.
* El rendimiento de la implementación. El rendimiento de la implementación se medirá en función los pasos necesarios para terminar la simulación.
* La calidad de la descripción de análisis, diseño e implementación del sistema multiagente, la elegancia de su diseño e implementación.


In [16]:
# Importamos las clases que se requieren para manejar los agentes (Agent) y su entorno (Model).
# Cada modelo puede contener múltiples agentes.
from mesa import Agent, Model 

# Debido a que necesitamos que todos los agentes inicien en una celda, elegimos ''MultiGrid''.
from mesa.space import SingleGrid, MultiGrid

# Con ''RandomActivation, hacemos que todos los agentes se activen ''al mismo tiempo''.
from mesa.time import SimultaneousActivation

# Haremos uso de ''DataCollector'' para obtener información de cada paso de la simulación.
from mesa.datacollection import DataCollector

# matplotlib lo usaremos crear una animación de cada uno de los pasos del modelo.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Importamos 'random' para generar números aleatorios.
import random

# Importamos los siguientes paquetes para el mejor manejo de valores numéricos.
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime

In [17]:
class BoxAgent(Agent):
    def __init__(self, id, model):
        super().__init__(id, model)

    def move(self):
        possible_cells = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False)
        
        empty_cells = [cell for cell in possible_cells if self.model.grid.is_cell_empty(cell)]

        if empty_cells:
            new_position = random.choice(empty_cells)
            self.model.grid.move_agent(self, new_position)

    def step(self):
        self.move()

In [18]:
def get_agents(Model):
    agents = np.zeros((Model.grid.width, Model.grid.height))
    for cell in Model.grid.coord_iter():
        cell_content, (x, y) = cell
        if len(cell_content) > 0:
            agents[x][y] = 1

    return agents

In [19]:
class BoxesModel(Model):
    def __init__(self, num_agents, width, height, steps):
        self.grid = MultiGrid(width, height, torus = False)
        self.floor = np.zeros((width, height))
        self.schedule = SimultaneousActivation(self)

        reporters = {"Agents": get_agents}

        self.datacollector = DataCollector(model_reporters=reporters)

        
        # Creamos los agentes en ubicaiones aleatorias.
        for i in range(num_agents):
            a = BoxAgent(i, self)
            self.schedule.add(a)
            x = random.randrange(width)
            y = random.randrange(height)
            self.grid.place_agent(a, (x, y))

        num_boxes = 200
        for _ in range(num_boxes):
            x = random.randrange(width)
            y = random.randrange(height)
            self.floor[x][y] = 1

    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()

In [20]:
WIDTH = 20
HEIGHT = 20
N = 5
STEPS = 100

model = BoxesModel(N, WIDTH, HEIGHT, STEPS)

for i in range(STEPS):
    model.step()

all_grid = model.datacollector.get_model_vars_dataframe()

In [21]:
fig, axs = plt.subplots(figsize=(5,5))

axs.set_xticks([])
axs.set_yticks([])

agents = all_grid.get('Agents')

data = agents

patch = axs.imshow(agents[0], cmap='Greys')
plt.close()

def animate(i):
    patch.set_data(data[i])

anim = animation.FuncAnimation(fig, animate, frames=STEPS, interval=100)

anim